In [1]:
import os
print(os.getcwd())
%load_ext autoreload
%autoreload 2

/home/udit/programs/Synaptic-Flow/Notebooks


In [2]:
import sys
sys.path.append("/home/udit/programs/Synaptic-Flow/")

In [3]:
args = {
"dataset" : "mnist", # ['mnist','cifar10','cifar100','tiny-imagenet','imagenet']
"model" : "fc", # ['fc','fc-orth','conv','conv-orth','strconv', ... <take rest from main.py>]
"model_class" : "default", # ['default','lottery','tinyimagenet','imagenet']
"dense_classifier" : False,
"pretrained" : False,
"optimizer" : "adam", # ['sgd','momentum','adam','rms']
"train_batch_size" : 128,
"test_batch_size" : 128,
"pre_epochs" : 0, # number of epochs to train before pruning
"post_epochs" : 20, # number of epochs to train after pruning
"lr" : 0.001,
"lr_drops" : [30, 60, 80],
"lr_drop_rate" : 0.1,
"weight_decay" : 1e-4 ,
"save" : False,
"scale" : 1,

# pruning args
"pruner" : "synflow", # ['rand','mag','snip','grasp','synflow']
"compression" : 1.0, # quotient of prunable non-zero prunable parameters before and after pruning (defaul"t: 1.0)
"prune_epochs" : 10, # number of iterations for scoring (defaul"t: 1)
"compression_schedule" : "exponential", # ['linear','exponential']
"mask_scope" : "global", # ['global','local']
"prune_dataset_ratio" : 10, # ratio of prune dataset size and number of classes (defaul"t: 10)'
"prune_batch_size" : 256, # input batch size for pruning (defaul"t: 256)
"prune_bias" : False,
"prune_batchnorm" : False,
"prune_residual" : False,
"reinitialize" : False, # whether to reinitialize weight parameters after pruning (defaul"t: False)
"pruner_list" : [], # list of pruning strategies for singleshot (defaul"t: [])
"prune_epoch_list" : [], # list of prune epochs for singleshot (defaul"t: [])
"compression_list" : [], # list of compression ratio exponents for singleshot/multishot (defaul"t: [])
"level_list" : [],
"experiment" : "prune-only", # ['example',"prune-only",'singleshot','multishot',
    # 'unit-conservation', 'layer-conservation','imp-conservation','schedule-conservation']
"expid" : "",
"result_dir" : "Results/data",
"gpu" : "0",
"workers" : 4,
"no_cuda" : False,
"seed" : 1,
"verbose" : False,
"save_pruned" : False,
"save_pruned_path" : "Results/pruned"
}

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from Utils import load
from Utils import generator
from Utils import metrics
from prune import *
import matplotlib.pyplot as plt

import os

def prune_only(args):
    print("Pruning only")
    
    ## Random Seed and Device ##
    torch.manual_seed(args["seed"])
    device = load.device(args["gpu"])
    shuffle = False
    ## Data ##
    print('Loading {} dataset.'.format(args["dataset"]))
    input_shape, num_classes = load.dimension(args["dataset"]) 
    prune_loader = load.dataloader(args["dataset"], args["prune_batch_size"], True, args["workers"]) #, 
                                   # args["prune_dataset_ratio"] * num_classes)
    
    train_loader = load.dataloader(args["dataset"], args["train_batch_size"], False, args["workers"])
    test_loader = load.dataloader(args["dataset"], args["test_batch_size"], False, args["workers"])
    
    
    ## Model, Loss, Optimizer ##
    print('Creating {}-{} model.'.format(args["model_class"], args["model"]))        

    model = load.model(args["model"], args["model_class"])(input_shape, 
                                                     num_classes, 
                                                     args["dense_classifier"], 
                                                     args["pretrained"]).to(device)
    loss = nn.CrossEntropyLoss()
    
        
    opt_class, opt_kwargs = load.optimizer(args["optimizer"])
    optimizer = opt_class(generator.parameters(model), lr=args["lr"], 
                          weight_decay=args["weight_decay"], **opt_kwargs)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=args["lr_drops"], 
                                                     gamma=args["lr_drop_rate"])

    
    #print(f"Kernel: {kernel}")
    
    ## Pre-Train ##
    #print('Pre-Train for {} epochs.'.format(args.pre_epochs))
    pre_result = train_eval_loop(model, loss, optimizer, scheduler, train_loader, 
                                 test_loader, device, 0, args["verbose"])
    
    
    ## Prune ##
    print('Pruning with {} for {} epochs.'.format(args["pruner"], args["prune_epochs"]))
    pruner = load.pruner(args["pruner"])(generator.masked_parameters(model, args["prune_bias"], 
                       args["prune_batchnorm"], args["prune_residual"]))
    sparsity = 10**(-float(args["compression"]))
    print("Sparsity: {}".format(sparsity))
    save_pruned_path = args["save_pruned_path"] + "/%s/%s/%s" % (args["model_class"], 
                                                                 args["model"], args["pruner"],)
    if (args["save_pruned"]):
        print("Saving pruned models to: %s" % (save_pruned_path, ))
        if not os.path.exists(save_pruned_path):
            os.makedirs(save_pruned_path)
    prune_loop(model, loss, pruner, prune_loader, device, sparsity, 
               args["compression_schedule"], args["mask_scope"], args["prune_epochs"], 
               args["reinitialize"], args["save_pruned"], save_pruned_path)
    
    prune_result = metrics.summary(model, pruner.scores,
                                   metrics.flop(model, input_shape, device),
                                   lambda p: generator.prunable(p, args["prune_batchnorm"], 
                                    args["prune_residual"]))
    total_params = int((prune_result['sparsity'] * prune_result['size']).sum())
    possible_params = prune_result['size'].sum()
    total_flops = int((prune_result['sparsity'] * prune_result['flops']).sum())
    possible_flops = prune_result['flops'].sum()
    print("Parameter Sparsity: {}/{} ({:.4f})".format(total_params, 
                                                      possible_params, total_params / possible_params))
    print("FLOP Sparsity: {}/{} ({:.4f})".format(total_flops, 
                                                 possible_flops, total_flops / possible_flops))
    
    ## Post-Train ##
    #print('Post-Training for {} epochs.'.format(args.post_epochs))
    post_result = train_eval_loop(model, loss, optimizer, scheduler, train_loader, 
                                  test_loader, device, args["post_epochs"], args["verbose"])
    
    print(save_pruned_path)
    ## Display Results ##
#     frames = [pre_result.head(1), post_result.head(1), post_result.tail(1)]
#     train_result = pd.concat(frames, keys=['Init.', 'Post-Prune', "Final"])

#     print("Train results:\n", train_result)
#     print("Prune results:\n", prune_result)
    if (args["save_result"]):
        save_result_path = args["save_pruned_path"] + "/%s/%s/%s" % (args["model_class"], 
                                                                 args["model"], args["pruner"],)
        if not os.path.exists(save_pruned_path):
            os.makedirs(save_result_path)
        post_result.to_csv(save_result_path + "/%s" % (args["dataset"] + "_" + str(args["seed"]) 
                                                                        + "_" + str(args["compression"]) + ".csv"))
    ## Save Results and Model ##
    if args["save"]:
        print('Saving results.')
        pre_result.to_pickle("{}/pre-train.pkl".format(args["result_dir"]))
        post_result.to_pickle("{}/post-train.pkl".format(args["result_dir"]))
        prune_result.to_pickle("{}/compression.pkl".format(args["result_dir"]))
        torch.save(model.state_dict(),"{}/model.pt".format(args["result_dir"]))
        torch.save(optimizer.state_dict(),"{}/optimizer.pt".format(args["result_dir"]))
        torch.save(pruner.state_dict(),"{}/pruner.pt".format(args["result_dir"]))
    
    # to change
    return post_result

In [5]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

def train(model, loss, optimizer, dataloader, device, epoch, verbose, log_interval=10):
    model.train()
    total = 0
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        model_hook(model, output, data, device)
        train_loss = loss(output, target)
        total += train_loss.item() * data.size(0)
        train_loss.backward()
        optimizer.step()
        if verbose & (batch_idx % log_interval == 0):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(dataloader.dataset),
                100. * batch_idx / len(dataloader), train_loss.item()))
        break
    return total / len(dataloader.dataset)

def eval(model, loss, dataloader, device, verbose):
    model.eval()
    total = 0
    correct1 = 0
    correct5 = 0
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            total += loss(output, target).item() * data.size(0)
            _, pred = output.topk(5, dim=1)
            correct = pred.eq(target.view(-1, 1).expand_as(pred))
            correct1 += correct[:,:1].sum().item()
            correct5 += correct[:,:5].sum().item()
    average_loss = total / len(dataloader.dataset)
    accuracy1 = 100. * correct1 / len(dataloader.dataset)
    accuracy5 = 100. * correct5 / len(dataloader.dataset)
    if verbose:
        print('Evaluation: Average loss: {:.4f}, Top 1 Accuracy: {}/{} ({:.2f}%)'.format(
            average_loss, correct1, len(dataloader.dataset), accuracy1))
    return average_loss, accuracy1, accuracy5

def train_eval_loop(model, loss, optimizer, scheduler, train_loader, test_loader, device, epochs, verbose):
    print("HELLO")
    test_loss, accuracy1, accuracy5 = eval(model, loss, test_loader, device, verbose)
    rows = [[np.nan, test_loss, accuracy1, accuracy5]]
    for epoch in tqdm(range(epochs)):
        train_loss = train(model, loss, optimizer, train_loader, device, epoch, verbose)
        test_loss, accuracy1, accuracy5 = eval(model, loss, test_loader, device, verbose)
        row = [train_loss, test_loss, accuracy1, accuracy5]
        scheduler.step()
        rows.append(row)
    columns = ['train_loss', 'test_loss', 'top1_accuracy', 'top5_accuracy']
    return pd.DataFrame(rows, columns=columns)

In [16]:
import copy

def model_hook(model, output, data, device):
    squared_copy = copy.deepcopy(model)
    
    for name, p in squared_copy.named_parameters():
        # print(p.data[p.data < 0])
        p.data = p.data ** 2
    
    input_dim = list(data.shape)[1:]
    # print(input_dim) # 1, 28, 28
    r_pk_ones = torch.ones([1] + input_dim).to(device)
    # print(r_pk_ones.shape) # 1, 1, 28, 28
    squared_copy(r_pk_ones).sum().backward()
    
    grad_sq_sum = 0
    for name, p in squared_copy.named_parameters():
        curr_grad_sq_sum = torch.sum(torch.clone(p.grad).detach())
        print(name, curr_grad_sq_sum)
        print("--"*5)
        grad_sq_sum += curr_grad_sq_sum
    print(f"Squared copy model grad sum: {grad_sq_sum}")
    
#     print("--"*20)
#     unsq_sum = 0
#     for _name, _p in model.named_parameters():
#         # print(_name)        
#         for name, p in unsquared_copy.named_parameters():
#             if _name == name:
#                 print(f"Processing {name}")
#                 p.data = torch.ones(p.data.shape).to(device)
#             else:
#                 p.data = p.data ** 2
        
#         unsq_grad_sum = unsquared_copy(r_pk_ones)
#         curr_sum = unsq_grad_sum.sum()
#         print(curr_sum)
#         unsq_sum += curr_sum
        
#         print("--"*5)
#         unsquared_copy = copy.deepcopy(model)        

#     print(f"Unsquared copy sum: {unsq_sum}")

In [17]:
args['post_epochs'] = 1
args['prune_epochs'] = 1
args['model'] = "wide-resnet20"
args['dataset'] = "cifar10"
args['lr'] = 0.001
args['optimizer'] = 'adam'
args['model_class'] = "lottery"
args['pruner'] = "synflow"
args['compression'] = 0.0
args['save_result'] = False
prune_only(args)

Pruning only
Loading cifar10 dataset.
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Creating lottery-wide-resnet20 model.
HELLO


0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

Pruning with synflow for 1 epochs.
Sparsity: 1.0


100%|██████████| 1/1 [00:00<00:00,  5.40it/s]
/home/udit/anaconda3/envs/synflow-py3.7/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


Parameter Sparsity: 1085066/1085066 (1.0000)
FLOP Sparsity: 163317002/163317002 (1.0000)
HELLO


  0%|          | 0/1 [00:00<?, ?it/s]

conv.weight tensor(0.1354, device='cuda:0')
----------
bn.weight tensor(0.0988, device='cuda:0')
----------
bn.bias tensor(0.2838, device='cuda:0')
----------
blocks.0.conv1.weight tensor(0.0021, device='cuda:0')
----------
blocks.0.bn1.weight tensor(1.0625e-05, device='cuda:0')
----------
blocks.0.bn1.bias tensor(5.7936e-05, device='cuda:0')
----------
blocks.0.conv2.weight tensor(0.3324, device='cuda:0')
----------
blocks.0.bn2.weight tensor(0.1330, device='cuda:0')
----------
blocks.0.bn2.bias tensor(0.2838, device='cuda:0')
----------
blocks.0.shortcut.weight tensor(0.0987, device='cuda:0')
----------
blocks.1.conv1.weight tensor(-1.1331, device='cuda:0')
----------
blocks.1.bn1.weight tensor(-8.3679e-07, device='cuda:0')
----------
blocks.1.bn1.bias tensor(0.1420, device='cuda:0')
----------
blocks.1.conv2.weight tensor(1.1911, device='cuda:0')
----------
blocks.1.bn2.weight tensor(-0.0379, device='cuda:0')
----------
blocks.1.bn2.bias tensor(0.2333, device='cuda:0')
----------
bl

100%|██████████| 1/1 [00:01<00:00,  1.12s/it]

Results/pruned/lottery/wide-resnet20/synflow


,train_loss,test_loss,top1_accuracy,top5_accuracy
0,NaN,77.322560,10.0,48.49
1,0.045062,1474.282265,10.0,50.00
